In [28]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow
import tqdm 
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from utils.data_utils import load_glove

wo = load_glove(50)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2273: character maps to <undefined>

In [27]:
import tensorflow as tf

tf.nn.dynamic_rnn(34, [])

TypeError: cell must be an instance of RNNCell